这是一个定制化的对股票估值的『毛估估』算法，主要是解决个人的两个需求，
1. 对不赢利的年轻公司进行估值；
2. 将不成熟的、成熟的、低增长的不同公司股票放在一起比较（虽然定量的比较一定是不太靠谱的）。

大体的做法是，
1. 基于通常的DCF方法中假设的『快速成长阶段』和『永续增长阶段』两个阶段，增加了一个『不稳定成长阶段』
2. 对『快速成长阶段』中的收益分配做了改变

具体细节可见下文代码及注释。

In [9]:
# 环境和包配置
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io

pd.set_option( 'display.precision',2)

In [10]:
# 全局参数
total_growth_years = 10 # 成长阶段总年数 = 不稳定成长阶段.年数 + 快速成长阶段.年数
terminal_years = 10 # 永续增长阶段.年数
terminal_growth = 0.04 # 永续增长阶段.增长率
discount = 0.1 # 折现率

pd.DataFrame([{
    "成长阶段总年数" : total_growth_years,
    "永续增长阶段.年数" : terminal_years,
    "永续增长阶段.增长率" : terminal_growth,
    "折现率" : discount
}])

def read_csv(csv_str):
    return pd.read_csv(io.StringIO(csv_str), comment='#', index_col=0, skipinitialspace=True)

# 估值计算过程, 针对传统 DCF 方法有一些变化, 但是总体而言并不关键, 关键的还是前面所说对于"不稳定成长阶段.期末EPS"的估计
def generalized_dcf(unstable_years, unstable_eps, growth_growth):
    """
    计算推广的DCF, 支持对不赢利或赢利不稳定的"年轻"公司的估值. 计算分为两个阶段, 见代码中注释。

    Parameters
    ----------
    
    """
    # 1. 估计 "快速成长阶段.期末EPS", 作为永续阶段计算的起点
    init_eps = unstable_eps / (1+growth_growth) ** unstable_years  # 期初估测EPS, 对于成熟公司来说等同于 EPS, 对于不成熟公司来说等于不稳定成长阶段.期末EPS对增长率进行折算
    growth_years = total_growth_years - unstable_years
    growth_eps = unstable_eps * (1+growth_growth)**growth_years

    # 2. 计算内在价值. 
    # 与传统 DCF 方法不同, 从利润留存/分红的角度来考虑, 在这个阶段为了保证快速成长, 绝大部分收益都应投入了扩大生产,
    # 所以并不计入快速成长阶段中的各年收益. 同时, 这样一来成长型行业与低增长型在逻辑上更有可比性(虽然一般不会这么干, 纯是心理作用).
    # 总的来说, 这个调整并不关键, 但因为不计成长阶段收益, 因此最终估值会相对其他一些估值方法偏小.
    q = (1+terminal_growth)/(1+discount) # 中间变量
    fair_values = growth_eps*q*(1-q**terminal_years) / ((1-q)*(1+discount)**total_growth_years)
    return init_eps, growth_years, growth_eps, fair_values

# 简易包装
def perform_valuation(data, current_prices):
    data = data.join(current_prices.rename("现价"))

    # 计算估值
    unstable_years = data["不稳定成长阶段.年数"]
    unstable_eps = data["不稳定成长阶段.期末EPS"]
    growth_growth = data["快速成长阶段.增长率"]/100
    init_eps, growth_years, growth_eps, fair_values = generalized_dcf(unstable_years, unstable_eps, growth_growth)
    names = data.index
    cols = ["期初估测EPS", "不稳定成长阶段.年数", "不稳定成长阶段.期末EPS", "快速成长阶段.年数", "快速成长阶段.增长率", "快速成长阶段.期末EPS", "内在价值"]
    output = pd.DataFrame(np.vstack((init_eps, unstable_years, unstable_eps, growth_years, growth_growth, growth_eps, fair_values)).T, index= names, columns=cols)

    # 代入现价, 比较安全边际
    prices = data["现价"]
    margin_of_safety = (fair_values - prices) / fair_values
    output["现价"] = prices
    output["安全边际%"] = np.array(margin_of_safety.T * 100)
    output["估测PE"] = prices / np.array(init_eps) # = 现价 / 期初EPS, 对于成熟公司来说等同于 PE
    return output


In [11]:
# 各公司特征数据
# ------
# 相较于传统的DCF计算, 在快速成长阶段和永续增长阶段之外, 增加了一个不稳定成长阶段, 主要是为了支持对不赢利或赢利不稳定的"年轻"公司的估值.
# 公平起见, 这个阶段与快速成长阶段共享总年数, 其期末EPS的估计需要根据个人对特定公司的理解来定, 也是这个毛估估方法的核心.
# -------
# 不稳定成长阶段.年数: 成熟公司设置为0; 未赢利公司考虑公司发展情况设置
# 不稳定成长阶段.期末EPS: 成熟公司设置为近期财报公布的扣非每股收益; 未赢利公司根据营收等预期估算, 在数据表中作相应说明
# 快速成长阶段.增长率: 成熟公司参考10年扣非每股收益增长率; 未赢利公司可参考行业内成熟企业的数值

data_202211 = read_csv(
"""
名称,   不稳定成长阶段.年数,    不稳定成长阶段.期末EPS,     快速成长阶段.增长率
网易,   0,                   7.66,                    16.6
腾讯,   0,                   21.86,                   20.0
快手,   5,                   12.54,                   16.6              # 假设20%营收增长, 行业20%的净利率, 公司最近12个月的每股收入为 25.20 HKD
京东,   2,                   21.81,                   16.6              # 近年50%的营收增长, 略低于同行costco的净利率 2.5%, 公司最近12个月的每股收入为 387.78 HKD
拼多多, 2,                   6.75,                    20.0              # 近年50%的营收(利润)增长, 公司最近12个月的扣非每股收益为 3 USD
美团,   2,                   13.11,                   20.0              # 假设近年30%的营收增长, 20%利润率, 公司最近12个月的每股收入为 38.78 HKD
苹果,   0,                   6.11,                    14.9
微软,   0,                   9.28,                    16.8
BRK,   0,                   26668.0,                 17.1
"""
)
data_202303 = read_csv(
"""
名称,   不稳定成长阶段.年数,    不稳定成长阶段.期末EPS,     快速成长阶段.增长率
网易,   0,                   6.91,                    16.6
腾讯,   0,                   22.02,                   20.0
快手,   4.75,                11.63,                   16.6              # 假设20%营收增长, 行业20%的净利率, 公司最近12个月的每股收入为 24.46 HKD
京东,   1.75,                19.47,                   16.6              # 近年50%的营收增长, 略低于同行costco的净利率 2.5%, 公司最近12个月的每股收入为 383.02 HKD
拼多多, 1.75,                6.53,                    20.0              # 近年50%的营收(利润)增长, 公司最近12个月的扣非每股收益为 3.21 USD
美团,   1.75,                12.59,                   20.0              # 假设近年30%的营收增长, 20%利润率, 公司最近12个月的每股收入为 39.76 HKD
苹果,   0,                   5.89,                    14.9
微软,   0,                   9.00,                    16.8
"""
)

In [12]:
# 计算打印各公司估值

prices = read_csv(
"""
名称,   20230216,   20230322
网易,   138.2,      134.0
腾讯,   382.6,      347.2
快手,   61.9,       54
京东,   212.6,      152.9
拼多多, 94.46,      78.93
美团,   148.3,      130.3
苹果,   155.33,     159.28
微软,   269.32,     273.78
BRK,   469045,     460515.03
""")
current_prices = prices["20230322"]

print("data_202211:")
output_202211 = perform_valuation(data_202211, current_prices)
display(output_202211)

print("data_202303:")
output_202303 = perform_valuation(data_202303, current_prices)
display(output_202303)

data_202211:


,期初估测EPS,不稳定成长阶段.年数,不稳定成长阶段.期末EPS,快速成长阶段.年数,快速成长阶段.增长率,快速成长阶段.期末EPS,内在价值,现价,安全边际%,估测PE
名称,,,,,,,,,,
网易,7.66,0.0,7.66,10.0,0.17,35.58,102.08,134.00,-31.27,17.49
腾讯,21.86,0.0,21.86,10.0,0.20,135.35,388.31,347.20,10.59,15.88
快手,5.82,5.0,12.54,5.0,0.17,27.03,77.54,54.00,30.36,9.28
京东,16.04,2.0,21.81,8.0,0.17,74.52,213.78,152.90,28.48,9.53
拼多多,4.69,2.0,6.75,8.0,0.20,29.02,83.27,78.93,5.21,16.84
美团,9.10,2.0,13.11,8.0,0.20,56.37,161.72,130.30,19.43,14.31
苹果,6.11,0.0,6.11,10.0,0.15,24.50,70.30,159.28,-126.57,26.07
微软,9.28,0.0,9.28,10.0,0.17,43.85,125.80,273.78,-117.62,29.50
BRK,26668.00,0.0,26668.00,10.0,0.17,129288.35,370917.47,460515.03,-24.16,17.27


data_202303:


,期初估测EPS,不稳定成长阶段.年数,不稳定成长阶段.期末EPS,快速成长阶段.年数,快速成长阶段.增长率,快速成长阶段.期末EPS,内在价值,现价,安全边际%,估测PE
名称,,,,,,,,,,
网易,6.91,0.00,6.91,10.00,0.17,32.10,92.08,134.00,-45.52,19.39
腾讯,22.02,0.00,22.02,10.00,0.20,136.34,391.15,347.20,11.24,15.77
快手,5.61,4.75,11.63,5.25,0.17,26.05,74.72,54.00,27.73,9.63
京东,14.88,1.75,19.47,8.25,0.17,69.12,198.31,152.90,22.90,10.27
拼多多,4.75,1.75,6.53,8.25,0.20,29.39,84.31,78.93,6.38,16.63
美团,9.15,1.75,12.59,8.25,0.20,56.66,162.55,130.30,19.84,14.24
苹果,5.89,0.00,5.89,10.00,0.15,23.62,67.77,159.28,-135.03,27.04
微软,9.00,0.00,9.00,10.00,0.17,42.53,122.01,273.78,-124.39,30.42
